# Full outcome model

In [1]:
# import required modules
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats

from classes.pathway import SSNAP_Pathway
from classes.clinical_outcome import Clinical_outcome

In [2]:
# # TEMPORARY 
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
hospital_performance = pd.read_csv('data/hospital_performance_thrombectomy.csv')

In [4]:
mrs_dists = pd.read_csv('data/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

In [5]:
mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


In [6]:
hospital_performance

,stroke_team,thrombolysis_rate,thrombectomy_rate,admissions,onset_known,known_arrival_within_4hrs,onset_arrival_mins_mu,onset_arrival_mins_sigma,scan_within_4_hrs,arrival_scan_arrival_mins_mu,...,onset_arrival_thrombectomy_mins_mu,onset_arrival_thrombectomy_mins_sigma,scan_within_8_hrs,arrival_scan_arrival_thrombectomy_mins_mu,arrival_scan_arrival_thrombectomy_mins_sigma,onset_scan_8_hrs,scan_puncture_8_hrs,scan_puncture_mins_mu,scan_puncture_mins_sigma,proportion_chosen_for_thrombectomy
0,Addenbrooke's Hospital,0.149184,0.026571,602.166667,0.590645,0.679007,4.636923,0.551945,0.948930,3.661057,...,4.849537,0.675364,0.980856,3.804003,0.820081,0.935706,0.960526,4.323977,1.264915,0.046626
1,Basildon University Hospital,0.132237,0.011990,486.500000,0.648167,0.572410,4.598469,0.531320,0.958449,2.467406,...,4.849547,0.691845,0.977957,2.820170,1.651319,0.921863,1.000000,5.344571,0.673775,0.023635
2,Blackpool Victoria Hospital,0.091938,0.012350,485.833333,0.448370,0.693191,4.477038,0.595180,0.941501,3.576595,...,4.694959,0.741009,0.975023,3.806109,0.959561,0.917457,1.000000,4.676798,0.972104,0.024819
3,Broomfield Hospital,0.104681,0.006635,452.166667,0.558791,0.605541,4.643075,0.543022,0.991285,2.859362,...,4.878630,0.683373,0.997382,3.049651,1.028392,0.949256,0.937500,5.188837,0.937691,0.014747
4,Calderdale Royal Hospital,0.135504,0.006040,634.666667,0.546218,0.645673,4.500197,0.608022,0.946389,2.708081,...,4.735035,0.747534,0.965244,3.023825,1.283449,0.931143,0.909091,5.260841,0.688688,0.014925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Wirral Arrowe Park Hospital,0.093588,0.006355,577.000000,0.740035,0.519906,4.530291,0.582275,0.969219,3.482265,...,4.782984,0.734571,0.986747,3.593126,0.790612,0.946276,1.000000,5.259824,0.306376,0.013548
79,Worthing Hospital,0.131726,0.004892,477.000000,0.638365,0.677614,4.560456,0.523377,0.953150,3.172186,...,4.780181,0.676788,0.979375,3.396003,1.152219,0.926630,0.916667,5.025557,0.685536,0.008798
80,Wycombe General Hospital,0.134859,0.014266,619.166667,0.723822,0.561547,4.596065,0.506447,0.975497,2.340029,...,4.857380,0.679072,0.984375,2.522393,1.400003,0.964021,1.000000,4.947447,0.909323,0.029089
81,Yeovil District Hospital,0.152151,0.009677,310.000000,0.665591,0.624394,4.687479,0.492452,0.972833,3.088504,...,4.916434,0.643097,0.983420,3.256842,0.999543,0.947313,0.941176,5.630844,0.495198,0.018910


## Run trials


# Set general model parameters
scenario_counter = 0
trials = 1

# Iterate through hospitals
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[0]
    hospital_data = hospital[1]
    run_data = hospital_data
    calibration = 1.0

    patients_per_run = int(run_data['admissions'])

    for trial in range(trials):

        patient_array = SSNAP_Pathway(hospital_name, hospital_data)
        patient_array.run_trial()

    print(stop, here, please)

patient_array.trial.keys()

patient_array.trial['scan_to_needle_mins'].data

patient_array.results_dataframe

print(stop, here, please)

In [7]:

# Set general model parameters
scenario_counter = 0
trials = 100

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = []
for column in outcome_results_columns:
    columns_here = [column + ending for ending in results_types]
    results_columns += columns_here
# Also store onset to needle time:
# results_columns += ['Onset_to_needle_(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    # 'onset_to_needle'
    ]

# Iterate through hospitals
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[0]
    hospital_data = hospital[1]
    run_data = hospital_data
    calibration = 1.0

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    number_of_patients = int(run_data['admissions'])

    for trial in range(trials):

        patient_array = SSNAP_Pathway(hospital_name, hospital_data)
        patient_array.run_trial()
        
        # Initiate the outcome model object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        for key in clinical_outcome.trial.keys():
            if key in patient_array.trial.keys():
                clinical_outcome.trial[key].data = patient_array.trial[key].data
        
        # Calculate outcomes:
        results, combo_results = clinical_outcome.calculate_outcomes()
        
        # mask = (
        #     patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
        # onset_to_needle_mins_masked = \
        #         patient_array.patient_array_onset_to_needle_mins[mask].mean()


        # Save scenario results to dataframe
        result = [
            np.mean(patient_array.trial['ivt_chosen_bool'].data)*100.0,
            results['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            # onset_to_needle_mins_masked
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        


    trial_result = []
    
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    # trial_result += [trial_df['onset_to_needle'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result
    
    # print(stop, here, please)

# Apply calibration
results_df['calibration'] = calibration
for col in list(results_df):
    if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
        results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


In [8]:
results_df

,Thrombolysis_rate_(%)_(median),Thrombolysis_rate_(%)_(low_5%),Thrombolysis_rate_(%)_(high_95%),Thrombolysis_rate_(%)_(mean),Thrombolysis_rate_(%)_(stdev),Thrombolysis_rate_(%)_(95ci),LVO_IVT_mean_shift_(median),LVO_IVT_mean_shift_(low_5%),LVO_IVT_mean_shift_(high_95%),LVO_IVT_mean_shift_(mean),...,LVO_MT_mean_shift_(mean),LVO_MT_mean_shift_(stdev),LVO_MT_mean_shift_(95ci),nLVO_IVT_mean_shift_(median),nLVO_IVT_mean_shift_(low_5%),nLVO_IVT_mean_shift_(high_95%),nLVO_IVT_mean_shift_(mean),nLVO_IVT_mean_shift_(stdev),nLVO_IVT_mean_shift_(95ci),calibration
0,15.116279,12.284053,17.284053,15.053156,1.636582,0.320764,0.001797,0.001364,0.002286,0.001805,...,0.004486,0.000850,0.000167,0.008904,0.007285,0.010494,0.008974,0.001025,0.000201,1.0
1,14.197531,12.139918,16.882716,14.251029,1.439771,0.282190,0.001859,0.001335,0.002448,0.001873,...,0.001110,0.000603,0.000118,0.008703,0.007293,0.010480,0.008751,0.000995,0.000195,1.0
2,9.072165,6.597938,10.938144,8.841237,1.369337,0.268385,0.001017,0.000583,0.001544,0.001044,...,0.001487,0.000590,0.000116,0.005317,0.003717,0.006704,0.005243,0.000907,0.000178,1.0
3,11.061947,8.617257,13.285398,10.951327,1.437064,0.281659,0.001461,0.000931,0.001948,0.001430,...,0.000615,0.000491,0.000096,0.006680,0.005315,0.008257,0.006716,0.000951,0.000186,1.0
4,14.195584,12.137224,16.403785,14.261830,1.286323,0.252115,0.002048,0.001719,0.002530,0.002069,...,0.000713,0.000435,0.000085,0.009323,0.007748,0.010592,0.009296,0.000930,0.000182,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,10.745234,8.838821,12.998267,10.783362,1.229833,0.241043,0.001400,0.000933,0.001772,0.001373,...,0.000667,0.000272,0.000053,0.006562,0.005292,0.008077,0.006576,0.000880,0.000172,1.0
79,13.941300,11.729560,16.771488,14.008386,1.599482,0.313493,0.001608,0.001136,0.002231,0.001651,...,0.000582,0.000400,0.000078,0.008173,0.006782,0.010063,0.008328,0.001017,0.000199,1.0
80,15.024233,12.915994,17.294023,15.058158,1.440441,0.282321,0.002131,0.001668,0.002620,0.002115,...,0.002572,0.000597,0.000117,0.009624,0.008247,0.011431,0.009655,0.000982,0.000192,1.0
81,16.129032,13.225806,19.354839,16.141935,1.889271,0.370290,0.001737,0.001247,0.002307,0.001743,...,0.000419,0.000515,0.000101,0.009203,0.007262,0.010954,0.009150,0.001150,0.000225,1.0


Print some info about the final trial:

In [9]:
print(clinical_outcome)

There are 730 patients and the base mRS distributions are: 
pre_stroke_nlvo array([0.582881, 0.745419, 0.848859, 0.951082, 0.993055, 1.      ,
       1.      ])
pre_stroke_nlvo_ivt_deaths array([0.57646931, 0.73721939, 0.83952155, 0.9406201 , 0.9821314 ,
       0.989     , 1.        ])
pre_stroke_lvo array([0.417894, 0.560853, 0.679283, 0.843494, 0.957269, 1.      ,
       1.      ])
pre_stroke_lvo_ivt_deaths array([0.40364381, 0.54172791, 0.65611945, 0.81473085, 0.92462613,
       0.9659    , 1.        ])
pre_stroke_lvo_mt_deaths array([0.40284982, 0.54066229, 0.65482881, 0.81312822, 0.92280732,
       0.964     , 1.        ])
no_treatment_nlvo array([0.19714427, 0.46      , 0.58003201, 0.70776826, 0.85567719,
       0.91770198, 1.        ])
no_effect_nlvo_ivt_deaths array([0.19727076, 0.46      , 0.57758322, 0.70225237, 0.84524353,
       0.90445421, 1.        ])
t0_treatment_nlvo_ivt array([0.42980772, 0.63      , 0.73821237, 0.84842726, 0.92918765,
       0.95630045, 1.        ])
n

In [10]:
results_df

,Thrombolysis_rate_(%)_(median),Thrombolysis_rate_(%)_(low_5%),Thrombolysis_rate_(%)_(high_95%),Thrombolysis_rate_(%)_(mean),Thrombolysis_rate_(%)_(stdev),Thrombolysis_rate_(%)_(95ci),LVO_IVT_mean_shift_(median),LVO_IVT_mean_shift_(low_5%),LVO_IVT_mean_shift_(high_95%),LVO_IVT_mean_shift_(mean),...,LVO_MT_mean_shift_(mean),LVO_MT_mean_shift_(stdev),LVO_MT_mean_shift_(95ci),nLVO_IVT_mean_shift_(median),nLVO_IVT_mean_shift_(low_5%),nLVO_IVT_mean_shift_(high_95%),nLVO_IVT_mean_shift_(mean),nLVO_IVT_mean_shift_(stdev),nLVO_IVT_mean_shift_(95ci),calibration
0,15.116279,12.284053,17.284053,15.053156,1.636582,0.320764,0.001797,0.001364,0.002286,0.001805,...,0.004486,0.000850,0.000167,0.008904,0.007285,0.010494,0.008974,0.001025,0.000201,1.0
1,14.197531,12.139918,16.882716,14.251029,1.439771,0.282190,0.001859,0.001335,0.002448,0.001873,...,0.001110,0.000603,0.000118,0.008703,0.007293,0.010480,0.008751,0.000995,0.000195,1.0
2,9.072165,6.597938,10.938144,8.841237,1.369337,0.268385,0.001017,0.000583,0.001544,0.001044,...,0.001487,0.000590,0.000116,0.005317,0.003717,0.006704,0.005243,0.000907,0.000178,1.0
3,11.061947,8.617257,13.285398,10.951327,1.437064,0.281659,0.001461,0.000931,0.001948,0.001430,...,0.000615,0.000491,0.000096,0.006680,0.005315,0.008257,0.006716,0.000951,0.000186,1.0
4,14.195584,12.137224,16.403785,14.261830,1.286323,0.252115,0.002048,0.001719,0.002530,0.002069,...,0.000713,0.000435,0.000085,0.009323,0.007748,0.010592,0.009296,0.000930,0.000182,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,10.745234,8.838821,12.998267,10.783362,1.229833,0.241043,0.001400,0.000933,0.001772,0.001373,...,0.000667,0.000272,0.000053,0.006562,0.005292,0.008077,0.006576,0.000880,0.000172,1.0
79,13.941300,11.729560,16.771488,14.008386,1.599482,0.313493,0.001608,0.001136,0.002231,0.001651,...,0.000582,0.000400,0.000078,0.008173,0.006782,0.010063,0.008328,0.001017,0.000199,1.0
80,15.024233,12.915994,17.294023,15.058158,1.440441,0.282321,0.002131,0.001668,0.002620,0.002115,...,0.002572,0.000597,0.000117,0.009624,0.008247,0.011431,0.009655,0.000982,0.000192,1.0
81,16.129032,13.225806,19.354839,16.141935,1.889271,0.370290,0.001737,0.001247,0.002307,0.001743,...,0.000419,0.000515,0.000101,0.009203,0.007262,0.010954,0.009150,0.001150,0.000225,1.0


In [11]:
results_df.to_csv('full_outcome_pathway_results_refactored.csv')